<a href="https://colab.research.google.com/github/Arcaneless/GAR_Efficient_Net_B0/blob/main/EfficientNet_GAR_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/My Drive/Colab Notebooks/socif/train.zip' /content
!cd /content/
!unzip train.zip

Archive:  train.zip
   creating: train/
  inflating: train/0001.jpg          
  inflating: train/0002.jpg          
  inflating: train/0003.jpg          
  inflating: train/0004.jpg          
  inflating: train/0005.jpg          
  inflating: train/0006.jpg          
  inflating: train/0007.jpg          
  inflating: train/0008.jpg          
  inflating: train/0009.jpg          
  inflating: train/0010.jpg          
  inflating: train/train.csv         


In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
import efficientnet.tfkeras as efn
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from pathlib import Path
import multiprocessing
import os

tf.enable_control_flow_v2()
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'


In [ ]:
# Building the model
efficient_net_lite_0 = "https://tfhub.dev/tensorflow/efficientnet/lite0/classification/1" 

# Input layer
input = tf.keras.layers.Input(shape=(224,224,3), name='input_dllm')
# This pre-trained layer is loaded from tensorflow hub
model_eff = hub.KerasLayer(efficient_net_lite_0, input_shape=[224,224,3], trainable=False)(input)
# This output is for gender
dense_G = tf.keras.layers.Dense(units=2, activation="softmax", name="pred_gender")(model_eff)
# This output is for age
dense_A = tf.keras.layers.Dense(units=101, activation="softmax", name="pred_age")(model_eff)
# This output is for race
dense_R = tf.keras.layers.Dense(units=3, activation="softmax", name="pred_race")(model_eff)
model = keras.Model(inputs=input, outputs=[dense_G, dense_A, dense_R], name='Ngo Dllm')

In [ ]:
# testing for efn only
model = efn.EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224,224,3), pooling='avg')

In [ ]:
model.summary()

Model: "Ngo Dllm"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_dllm (InputLayer)         [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      (None, 1000)         4694024     input_dllm[0][0]                 
__________________________________________________________________________________________________
pred_gender (Dense)             (None, 2)            2002        keras_layer_1[0][0]              
__________________________________________________________________________________________________
pred_age (Dense)                (None, 101)          101101      keras_layer_1[0][0]              
___________________________________________________________________________________________

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam
def get_optimizer(name="adam", learning_rate=0.001):
    if name == "sgd":
        return SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    elif name == "adam":
        return Adam(lr=learning_rate)
    else:
        raise ValueError("optimizer name should be 'sgd' or 'adam'")


def get_scheduler(epoches=30, learning_rate=0.001):
    class Schedule:
        def __init__(self, nb_epochs, initial_lr):
            self.epochs = nb_epochs
            self.initial_lr = initial_lr

        def __call__(self, epoch_idx):
            if epoch_idx < self.epochs * 0.25:
                return self.initial_lr
            elif epoch_idx < self.epochs * 0.50:
                return self.initial_lr * 0.2
            elif epoch_idx < self.epochs * 0.75:
                return self.initial_lr * 0.04
            return self.initial_lr * 0.008
    return Schedule(epoches, learning_rate)

In [ ]:
csv_path = "./train/train.csv"
dataset = pd.read_csv(csv_path, header=None)
X = dataset[0].apply(lambda p: cv2.imread("./train/"+p))
X = X.apply(lambda img: cv2.resize(img, (224, 224)))
X = np.array(X.to_list(), dtype=np.float16)

In [ ]:
y = dataset.loc[:,1:].to_numpy()

In [ ]:
import tensorflow.keras as keras

# Compiling the model
opt = get_optimizer()
scheduler = get_scheduler()
model.compile(optimizer=opt, loss=["sparse_categorical_crossentropy", 
                "sparse_categorical_crossentropy",
                "sparse_categorical_crossentropy"],metrics=['accuracy'])

checkpoint_dir = Path("./checkpoint")
checkpoint_dir.mkdir(exist_ok=True)
filename = "_".join(["EfficientNetB3","224","weights.{epoch:02d}-{loss:.2f}.hdf5"])
callbacks= [
    LearningRateScheduler(schedule=scheduler),
    ModelCheckpoint(str(checkpoint_dir) + "/" + filename,
                    monitor="loss",
                    verbose=1,
                    save_best_only=True,
                    mode="auto"),
]

In [ ]:
model.fit(X, y=[y[:,0], y[:,1], y[:,2]], epochs=10, batch_size=10, callbacks=callbacks)

Train on 10 samples
Epoch 1/10

Epoch 00001: loss improved from inf to 8.87203, saving model to checkpoint/EfficientNetB3_224_weights.01-8.87.hdf5
10/10 [==============================] - 2s 220ms/sample - loss: 8.8720 - pred_gender_loss: 1.2148 - pred_age_loss: 5.9130 - pred_race_loss: 1.7442 - pred_gender_acc: 0.6000 - pred_age_acc: 0.0000e+00 - pred_race_acc: 0.3000
Epoch 2/10

Epoch 00002: loss improved from 8.87203 to 6.34907, saving model to checkpoint/EfficientNetB3_224_weights.02-6.35.hdf5
10/10 [==============================] - 0s 16ms/sample - loss: 6.3491 - pred_gender_loss: 0.9335 - pred_age_loss: 4.2209 - pred_race_loss: 1.1947 - pred_gender_acc: 0.6000 - pred_age_acc: 0.3000 - pred_race_acc: 0.3000
Epoch 3/10

Epoch 00003: loss improved from 6.34907 to 4.65237, saving model to checkpoint/EfficientNetB3_224_weights.03-4.65.hdf5
10/10 [==============================] - 0s 15ms/sample - loss: 4.6524 - pred_gender_loss: 0.6896 - pred_age_loss: 2.9365 - pred_race_loss: 1.0262

In [ ]:
model.load_weights("/content/checkpoint/EfficientNetB3_224_weights.10-2.82.hdf5")

In [ ]:
model.summary()

Model: "Ngo Dllm"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_dllm (InputLayer)         [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      (None, 1000)         8273768     input_dllm[0][0]                 
__________________________________________________________________________________________________
pred_gender (Dense)             (None, 2)            2002        keras_layer_2[0][0]              
__________________________________________________________________________________________________
pred_age (Dense)                (None, 101)          101101      keras_layer_2[0][0]              
___________________________________________________________________________________________

In [ ]:
from tensorflow import keras
# check input and output names
in_names = [x.op.name for x in model.inputs]
out_names = [x.op.name for x in model.outputs]
print(in_names)
print(out_names)

['input_dllm_3']
['pred_gender_3/Softmax', 'pred_age_3/Softmax', 'pred_race_3/Softmax']


In [ ]:
import keras.backend as K

In [ ]:
# Saved the model to .h5 format, P.S. the model must not contain subclass, it should be functional or sequential
tf.keras.models.save_model(model, '/content/saved_model.h5')

In [ ]:
# not working
# converter = tf.compat.v1.lite.TFLiteConverter.from_session(
#   K.get_session(), input_tensors=model.inputs, output_tensors=model.outputs)
# tflite_model = converter.convert()
# open("converted_model.tflite", "wb").write(tflite_model)

In [ ]:
# Convert .h5 file to .tflite
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(
    model_file='/content/saved_model.h5',
    input_arrays=['input_dllm'],
    output_arrays=['pred_gender/Softmax', 'pred_age/Softmax', 'pred_race/Softmax'],
    input_shapes={"input_dllm" : [None, 224,224,3]},
    custom_objects={'KerasLayer': hub.KerasLayer}
)
tflite_model = converter.convert()
open("b0_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Froze 253 variables.


INFO:tensorflow:Froze 253 variables.


INFO:tensorflow:Converted 253 variables to const ops.


INFO:tensorflow:Converted 253 variables to const ops.


18978712

In [ ]:
# testing for layer itself
# converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(
#     model_file='/content/saved_model.h5',
#     input_arrays=['input_2'],
#     output_arrays=['avg_pool/Mean'],
#     input_shapes={"input_2" : [None, 224,224,3]},
#     #custom_objects={'KerasLayer': hub.KerasLayer}
# )
# tflite_model = converter.convert()
# open("test_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Froze 494 variables.


INFO:tensorflow:Froze 494 variables.


INFO:tensorflow:Converted 494 variables to const ops.


INFO:tensorflow:Converted 494 variables to const ops.


42705064

In [ ]:
print([i.op.name for i in model.inputs])
print([out.op.name for out in model.outputs])

['input_dllm_1']
['pred_gender_1/Softmax', 'pred_age_1/Softmax', 'pred_race_1/Softmax']


In [ ]:
!cp '/content/b0_model.tflite' '/content/drive/My Drive/Colab Notebooks'